# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database Schema

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database **data.sqlite**.

In [1]:
# Your code here; import the necessary packages
import sqlite3
import pandas as pd
# Your code here; create the connection and cursor
conn = sqlite3.Connection('data.sqlite')
cur = conn.cursor()

## Write an Equivalent Query using a Subquery

```SQL
SELECT customerNumber,
       contactLastName,
       contactFirstName
       FROM customers
       JOIN orders 
       USING(customerNumber)
       WHERE orderDate = '2003-01-31';
```

In [2]:
# Your code here; use a subquery. No join is necessary 
cur.execute("""
            SELECT customerNumber, contactLastName, contactFirstName
            FROM customers
            WHERE customerNumber IN (SELECT customerNumber
                                     FROM orders
                                     WHERE orderDate = '2003-01-31');
                                     """).fetchall()

[(141, 'Freyre', 'Diego ')]

In [3]:
cur.execute("""
            SELECT customerNumber, contactLastName, contactFirstName
            FROM customers
            WHERE customerNumber IN (SELECT customerNumber
                                     FROM orders
                                     WHERE orderDate = '2003-01-31');
                                     """)
df = pd.DataFrame(cur.fetchall())
df.columns = list(x[0] for x in cur.description)
df

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [4]:
# Your code here
cur.execute("""
            SELECT productName, COUNT(orderNumber) AS numberOrders, 
                   SUM(quantityOrdered) AS totalSoldItems
            FROM products
            JOIN orderdetails
            USING(productCode)
            GROUP BY productName
            ORDER BY totalSoldItems DESC
            ;""").fetchall()

[('1992 Ferrari 360 Spider red', 53, 1808),
 ('1937 Lincoln Berline', 28, 1111),
 ('American Airlines: MD-11S', 28, 1085),
 ('1941 Chevrolet Special Deluxe Cabriolet', 28, 1076),
 ('1930 Buick Marquette Phaeton', 28, 1074),
 ('1940s Ford truck', 28, 1061),
 ('1969 Harley Davidson Ultimate Chopper', 28, 1057),
 ('1957 Chevy Pickup', 28, 1056),
 ('1964 Mercedes Tour Bus', 28, 1053),
 ('1956 Porsche 356A Coupe', 27, 1052),
 ('Corsair F4U ( Bird Cage)', 28, 1051),
 ('F/A 18 Hornet 1/72', 28, 1047),
 ('1980s Black Hawk Helicopter', 28, 1040),
 ('1913 Ford Model T Speedster', 28, 1038),
 ('1997 BMW R 1100 S', 28, 1033),
 ('1972 Alfa Romeo GTA', 28, 1030),
 ('1962 Volkswagen Microbus', 28, 1029),
 ('2002 Suzuki XREO', 28, 1028),
 ('The USS Constitution Ship', 27, 1020),
 ('2001 Ferrari Enzo', 27, 1019),
 ('1960 BSA Gold Star DBD34', 28, 1015),
 ('1997 BMW F650 ST', 28, 1014),
 ('1957 Corvette Convertible', 27, 1013),
 ('18th century schooner', 27, 1011),
 ('1900s Vintage Tri-Plane', 28, 1009)

In [5]:
cur.execute("""
            SELECT productName, COUNT(orderNumber) AS numberOrders, 
                   SUM(quantityOrdered) AS totalSoldItems
            FROM products
            JOIN orderdetails
            USING(productCode)
            GROUP BY productName
            ORDER BY totalSoldItems DESC
            ;""")
df = pd.DataFrame(cur.fetchall())
df.columns = list(x[0] for x in cur.description)
print(df.tail())
df

                          productName  numberOrders  totalSoldItems
104      1999 Indy 500 Monte Carlo SS            25             855
105                1911 Ford Town Car            25             832
106  1936 Mercedes Benz 500k Roadster            25             824
107        1970 Chevy Chevelle SS 454            25             803
108             1957 Ford Thunderbird            24             767


,productName,numberOrders,totalSoldItems
0,1992 Ferrari 360 Spider red,53,1808
1,1937 Lincoln Berline,28,1111
2,American Airlines: MD-11S,28,1085
3,1941 Chevrolet Special Deluxe Cabriolet,28,1076
4,1930 Buick Marquette Phaeton,28,1074
...,...,...,...
104,1999 Indy 500 Monte Carlo SS,25,855
105,1911 Ford Town Car,25,832
106,1936 Mercedes Benz 500k Roadster,25,824
107,1970 Chevy Chevelle SS 454,25,803


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

### A quick note on the SQL  `SELECT DISTINCT` statement:

The `SELECT DISTINCT` statement is used to return only distinct values in the specified column. In other words, it removes the duplicate values in the column from the result set.

Inside a table, a column often contains many duplicate values; and sometimes you only want to list the unique values. If you apply the `DISTINCT` clause to a column that has `NULL`, the `DISTINCT` clause will keep only one NULL and eliminates the other. In other words, the DISTINCT clause treats all `NULL` “values” as the same value.

In [6]:
# Your code here:
# Hint: because one of the tables we'll be joining has duplicate customer numbers, 
# you should use DISTINCT
cur.execute("""
            SELECT productName, COUNT(DISTINCT customerNumber) AS numberCustomers, 
                   SUM(quantityOrdered) AS totalSoldItems
            FROM products
            JOIN orderdetails
            USING(productCode)
            JOIN orders
            USING(orderNumber)
            GROUP BY productName
            ORDER BY numberCustomers DESC
            ;""").fetchall()

[('1992 Ferrari 360 Spider red', 40, 1808),
 ('Boeing X-32A JSF', 27, 960),
 ('1972 Alfa Romeo GTA', 27, 1030),
 ('1952 Alpine Renault 1300', 27, 961),
 ('1934 Ford V8 Coupe', 27, 985),
 ('HMS Bounty', 26, 897),
 ('Corsair F4U ( Bird Cage)', 26, 1051),
 ('American Airlines: MD-11S', 26, 1085),
 ('1999 Yamaha Speed Boat', 26, 966),
 ('1996 Moto Guzzi 1100i', 26, 999),
 ('1995 Honda Civic', 26, 917),
 ('1980s Black Hawk Helicopter', 26, 1040),
 ('1969 Harley Davidson Ultimate Chopper', 26, 1057),
 ('1941 Chevrolet Special Deluxe Cabriolet', 26, 1076),
 ('1940 Ford Delivery Sedan', 26, 923),
 ('1937 Horch 930V Limousine', 26, 884),
 ('1913 Ford Model T Speedster', 26, 1038),
 ('1900s Vintage Bi-Plane', 26, 940),
 ('18th Century Vintage Horse Carriage', 26, 907),
 ('America West Airlines B757-200', 25, 984),
 ('ATA: B757-300', 25, 973),
 ('2003 Harley-Davidson Eagle Drag Bike', 25, 985),
 ('2002 Suzuki XREO', 25, 1028),
 ('1997 BMW R 1100 S', 25, 1033),
 ('1993 Mazda RX-7', 25, 947),
 ('19

In [7]:
cur.execute("""
            SELECT productName, COUNT(DISTINCT customerNumber) AS numberCustomers, 
                   SUM(quantityOrdered) AS totalSoldItems
            FROM products
            JOIN orderdetails
            USING(productCode)
            JOIN orders
            USING(orderNumber)
            GROUP BY productName
            ORDER BY numberCustomers DESC
            ;""")
df = pd.DataFrame(cur.fetchall())
df.columns = list(x[0] for x in cur.description)
print(df.tail(),'\n')
print(df.head())
df

                             productName  numberCustomers  totalSoldItems
104  1958 Chevy Corvette Limited Edition               19             983
105                  2002 Chevy Corvette               18             894
106            1969 Chevrolet Camaro Z28               18             870
107                    1952 Citroen-15CV               18             873
108                   1949 Jaguar XK 120               18             949 

                   productName  numberCustomers  totalSoldItems
0  1992 Ferrari 360 Spider red               40            1808
1             Boeing X-32A JSF               27             960
2          1972 Alfa Romeo GTA               27            1030
3     1952 Alpine Renault 1300               27             961
4           1934 Ford V8 Coupe               27             985


,productName,numberCustomers,totalSoldItems
0,1992 Ferrari 360 Spider red,40,1808
1,Boeing X-32A JSF,27,960
2,1972 Alfa Romeo GTA,27,1030
3,1952 Alpine Renault 1300,27,961
4,1934 Ford V8 Coupe,27,985
...,...,...,...
104,1958 Chevy Corvette Limited Edition,19,983
105,2002 Chevy Corvette,18,894
106,1969 Chevrolet Camaro Z28,18,870
107,1952 Citroen-15CV,18,873


## Select the Employee Number, First Name, Last Name, City (of the office), and Office Code of the Employees Who Sold Products Which Have Been Ordered by Less Then 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [8]:
# Your code here
cur.execute("""
            SELECT DISTINCT e.employeeNumber, e.firstName, e.lastName, o.city, e.officeCode
            FROM employees e
            JOIN offices o
            USING(officeCode)
            JOIN customers c
            ON e.employeeNumber = c.salesRepEmployeeNumber
            JOIN (orders)
            USING(customerNumber)
            JOIN orderdetails
            USING (orderNumber)
            WHERE productCode in (SELECT productCode
                                  FROM products
                                  JOIN orderdetails
                                  USING(productCode)
                                  JOIN orders
                                  USING(orderNumber)
                                  GROUP BY productCode
                                  HAVING COUNT(DISTINCT customerNumber) < 20);
                                  """).fetchall()

[(1370, 'Gerard', 'Hernandez', 'Paris', 4),
 (1501, 'Larry', 'Bott', 'London', 7),
 (1337, 'Loui', 'Bondur', 'Paris', 4),
 (1166, 'Leslie', 'Thompson', 'San Francisco', 1),
 (1286, 'Foon Yue', 'Tseng', 'NYC', 3),
 (1612, 'Peter', 'Marsh', 'Sydney', 6),
 (1611, 'Andy', 'Fixter', 'Sydney', 6),
 (1401, 'Pamela', 'Castillo', 'Paris', 4),
 (1621, 'Mami', 'Nishi', 'Tokyo', 5),
 (1323, 'George', 'Vanauf', 'NYC', 3),
 (1165, 'Leslie', 'Jennings', 'San Francisco', 1),
 (1702, 'Martin', 'Gerard', 'Paris', 4),
 (1216, 'Steve', 'Patterson', 'Boston', 2),
 (1188, 'Julie', 'Firrelli', 'Boston', 2),
 (1504, 'Barry', 'Jones', 'London', 7)]

In [9]:
cur.execute("""
            SELECT DISTINCT e.employeeNumber, e.firstName, e.lastName, o.city, e.officeCode
            FROM employees e
            JOIN offices o
            USING(officeCode)
            JOIN customers c
            ON e.employeeNumber = c.salesRepEmployeeNumber
            JOIN (orders)
            USING(customerNumber)
            JOIN orderdetails
            USING (orderNumber)
            WHERE productCode in (SELECT productCode
                                  FROM products
                                  JOIN orderdetails
                                  USING(productCode)
                                  JOIN orders
                                  USING(orderNumber)
                                  GROUP BY productCode
                                  HAVING COUNT(DISTINCT customerNumber) < 20);
                                  """)
df = pd.DataFrame(cur.fetchall())
df.columns = list(x[0] for x in cur.description)
df

,employeeNumber,firstName,lastName,city,officeCode
0,1370,Gerard,Hernandez,Paris,4
1,1501,Larry,Bott,London,7
2,1337,Loui,Bondur,Paris,4
3,1166,Leslie,Thompson,San Francisco,1
4,1286,Foon Yue,Tseng,NYC,3
5,1612,Peter,Marsh,Sydney,6
6,1611,Andy,Fixter,Sydney,6
7,1401,Pamela,Castillo,Paris,4
8,1621,Mami,Nishi,Tokyo,5
9,1323,George,Vanauf,NYC,3


## Select the Employee Number, First Name, Last Name, and Number of Customers for Employees Whose Customers Have an Average Credit Limit of Over 15K

In [10]:
# Your code here
cur.execute("""
            SELECT DISTINCT e.employeeNumber, e.firstName, e.lastName, 
                   COUNT(DISTINCT c.customerNumber) AS num_of_customers,
                   AVG(c.creditLimit) AS avg_creditlimit
            FROM employees e
            JOIN customers c
            ON e.employeeNumber = c.salesRepEmployeeNumber
            GROUP BY e.employeeNumber, e.firstName, e.lastName
            HAVING avg_creditlimit > 15000;""").fetchall()

[(1165, 'Leslie', 'Jennings', 6, 100433.33333333333),
 (1166, 'Leslie', 'Thompson', 6, 65266.666666666664),
 (1188, 'Julie', 'Firrelli', 6, 73916.66666666667),
 (1216, 'Steve', 'Patterson', 6, 81533.33333333333),
 (1286, 'Foon Yue', 'Tseng', 7, 66614.28571428571),
 (1323, 'George', 'Vanauf', 8, 80887.5),
 (1337, 'Loui', 'Bondur', 6, 86233.33333333333),
 (1370, 'Gerard', 'Hernandez', 7, 91785.71428571429),
 (1401, 'Pamela', 'Castillo', 10, 81340.0),
 (1501, 'Larry', 'Bott', 8, 91187.5),
 (1504, 'Barry', 'Jones', 9, 85511.11111111111),
 (1611, 'Andy', 'Fixter', 5, 86060.0),
 (1612, 'Peter', 'Marsh', 5, 92080.0),
 (1621, 'Mami', 'Nishi', 5, 83900.0),
 (1702, 'Martin', 'Gerard', 6, 71916.66666666667)]

In [11]:
cur.execute("""
            SELECT DISTINCT e.employeeNumber, e.firstName, e.lastName, 
                   COUNT(DISTINCT c.customerNumber) AS num_of_customers,
                   AVG(c.creditLimit) AS avg_creditlimit
            FROM employees e
            JOIN customers c
            ON e.employeeNumber = c.salesRepEmployeeNumber
            GROUP BY e.employeeNumber, e.firstName, e.lastName
            HAVING avg_creditlimit > 15000;""")
df = pd.DataFrame(cur.fetchall())
df.columns = list(x[0] for x in cur.description)
df

,employeeNumber,firstName,lastName,num_of_customers,avg_creditlimit
0,1165,Leslie,Jennings,6,100433.333333
1,1166,Leslie,Thompson,6,65266.666667
2,1188,Julie,Firrelli,6,73916.666667
3,1216,Steve,Patterson,6,81533.333333
4,1286,Foon Yue,Tseng,7,66614.285714
5,1323,George,Vanauf,8,80887.500000
6,1337,Loui,Bondur,6,86233.333333
7,1370,Gerard,Hernandez,7,91785.714286
8,1401,Pamela,Castillo,10,81340.000000
9,1501,Larry,Bott,8,91187.500000


## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!